In [1]:
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://download.pytorch.org/whl/cu117


In [2]:
%pip install transformers requests beatifulsoup4 pandas numpy

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement beatifulsoup4 (from versions: none)
ERROR: No matching distribution found for beatifulsoup4

[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
%pip install html5lib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


d:\Software\Python\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from urllib.request import urlopen
from selenium import webdriver

d:\Software\Python\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

In [3]:
# Set up the Selenium driver
driver = webdriver.Firefox()

# Navigate to the page you want to scrape
url = "https://www.expedia.com/things-to-do/budapest-all-in-one-walking-tour-with-cafe-stop.a844683.activity-details?endDate=2023-05-21&filter.categories=ToursDayTrips%7CToursDayTrips%7CCityTours&location=Budapest%20%28and%20vicinity%29%2C%20Hungary&rid=179994&sort=RECOMMENDED&startDate=2023-05-20&swp=on"
driver.get(url)

# Wait for the page to load
driver.implicitly_wait(10)

# Get the page's HTML content
html = driver.page_source

# Parse the HTML using BeautifulSoup
soup = BeautifulSoup(html, 'lxml')



In [4]:
# Find the comments on the page
comments = soup.find_all('div', class_='uitk-text overflow-wrap uitk-type-300 uitk-text-default-theme')

# Extract and print the text of each comment
for comment in comments:
    comment_text = comment.get_text()
    print(comment_text)

# This is just here to check the comments

Good value for money. Clear commentary and friendly staff. Comfortable cruise. Clean . Great views. I would definitely use this company again.
Great experience, the views and staff exceptional, and a glass of champagne to enjoy while learning the history of this beautiful city. Best money spent in Budapest.
Anche se il tempo, non ci ha accompagnato perché pioveva, è stata un'esperienza molto bella. Personale gentile, imbarcazione splendida, tour con veduta a 360°e.... Consiglio di farla, scegliendo questa compagnia
This was an amazing trip and I would 100% recommend the even trip as the city looks beautiful when it is all lit up. The boat was lovely and clean and we also enjoyed a lovely beer.
Det var en helt fantastisk tur og virkelig penge værd, vi nøde og meget nemt at komme til ikke sidste gang


In [5]:
# Find the comments on the page
comments = soup.find_all('div', class_='uitk-text overflow-wrap uitk-type-300 uitk-text-default-theme')

# Create an empty list to store the comments
comments_list = []

# Extract and append the text of each comment to the list
for comment in comments:
    comment_text = comment.get_text()
    comments_list.append(comment_text)
print(comments_list[1])


Great experience, the views and staff exceptional, and a glass of champagne to enjoy while learning the history of this beautiful city. Best money spent in Budapest.


In [6]:
tokens=tokenizer.encode(comment_text, return_tensors="pt")

result = model(tokens)

result.logits

int(torch.argmax(result.logits))+1
print(comment_text)

Det var en helt fantastisk tur og virkelig penge værd, vi nøde og meget nemt at komme til ikke sidste gang


In [7]:
# Iterate over the comments in the comments_list array
for comment_text in comments_list:
    # Tokenize the comment_text
    tokens = tokenizer.encode(comment_text, return_tensors="pt")

    # Pass the tokens to the model
    result = model(tokens)

    # Get the predicted sentiment from the model's output logits
    sentiment = int(torch.argmax(result.logits)) + 1

    # Print the comment and its predicted sentiment
    print(f"Comment: {comment_text}")
    print(f"Predicted sentiment: {sentiment}")

Comment: Good value for money. Clear commentary and friendly staff. Comfortable cruise. Clean . Great views. I would definitely use this company again.
Predicted sentiment: 4
Comment: Great experience, the views and staff exceptional, and a glass of champagne to enjoy while learning the history of this beautiful city. Best money spent in Budapest.
Predicted sentiment: 5
Comment: Anche se il tempo, non ci ha accompagnato perché pioveva, è stata un'esperienza molto bella. Personale gentile, imbarcazione splendida, tour con veduta a 360°e.... Consiglio di farla, scegliendo questa compagnia
Predicted sentiment: 5
Comment: This was an amazing trip and I would 100% recommend the even trip as the city looks beautiful when it is all lit up. The boat was lovely and clean and we also enjoyed a lovely beer.
Predicted sentiment: 5
Comment: Det var en helt fantastisk tur og virkelig penge værd, vi nøde og meget nemt at komme til ikke sidste gang
Predicted sentiment: 5


In [8]:
# Find the user-rated values on the page
user_scores = soup.find_all('span', class_='is-visually-hidden')

# Create an empty list to store the user-rated values
user_scores_list = []

# Extract and append the first character of each user score to the list, skipping the first instance
for score in user_scores[-5:]:
    score_text = score.get_text()
    if score_text:
        try:
            user_score = float(score_text[0])
            user_scores_list.append(user_score)
        except ValueError:
            # The first character could not be converted to a float, so skip it
            pass
# Iterate over the user scores in the user_scores_list array
for user_score in user_scores_list:
    # Print the user score
    print(user_score)

5.0
5.0
4.0
5.0
5.0


In [9]:
# Iterate over the comments and user scores
for comment_text, user_score in zip(comments_list, user_scores_list):
    # Tokenize the comment_text
    tokens = tokenizer.encode(comment_text, return_tensors="pt")

    # Pass the tokens to the model
    result = model(tokens)

    # Get the predicted sentiment from the model's output logits
    sentiment = int(torch.argmax(result.logits)) + 1

    # Print the comment, its predicted sentiment, and the user-rated value
    print(f"Comment: {comment_text}")
    print(f"Predicted sentiment: {sentiment}")
    print(f"User-rated value: {user_score}")

    # Compare the predicted sentiment with the user-rated value
    if sentiment == user_score:
        print("The predicted sentiment matches the user-rated value")
    else:
        print("The predicted sentiment does not match the user-rated value")

Comment: Good value for money. Clear commentary and friendly staff. Comfortable cruise. Clean . Great views. I would definitely use this company again.
Predicted sentiment: 4
User-rated value: 5.0
The predicted sentiment does not match the user-rated value
Comment: Great experience, the views and staff exceptional, and a glass of champagne to enjoy while learning the history of this beautiful city. Best money spent in Budapest.
Predicted sentiment: 5
User-rated value: 5.0
The predicted sentiment matches the user-rated value
Comment: Anche se il tempo, non ci ha accompagnato perché pioveva, è stata un'esperienza molto bella. Personale gentile, imbarcazione splendida, tour con veduta a 360°e.... Consiglio di farla, scegliendo questa compagnia
Predicted sentiment: 5
User-rated value: 4.0
The predicted sentiment does not match the user-rated value
Comment: This was an amazing trip and I would 100% recommend the even trip as the city looks beautiful when it is all lit up. The boat was lovel